Heavily based on
https://github.com/nicknochnack/ActionDetectionforSignLanguage

# 1. Import and Install Dependencies

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!unzip /content/drive/MyDrive/UCF101.zip -d UCF-101

Streaming output truncated to the last 5000 lines.
  inflating: UCF-101/UCF-101/PlayingGuitar/v_PlayingGuitar_g01_c02.avi  
  inflating: UCF-101/UCF-101/PlayingGuitar/v_PlayingGuitar_g01_c03.avi  
  inflating: UCF-101/UCF-101/PlayingGuitar/v_PlayingGuitar_g01_c04.avi  
  inflating: UCF-101/UCF-101/PlayingGuitar/v_PlayingGuitar_g01_c05.avi  
  inflating: UCF-101/UCF-101/PlayingGuitar/v_PlayingGuitar_g01_c06.avi  
  inflating: UCF-101/UCF-101/PlayingGuitar/v_PlayingGuitar_g02_c01.avi  
  inflating: UCF-101/UCF-101/PlayingGuitar/v_PlayingGuitar_g02_c02.avi  
  inflating: UCF-101/UCF-101/PlayingGuitar/v_PlayingGuitar_g02_c03.avi  
  inflating: UCF-101/UCF-101/PlayingGuitar/v_PlayingGuitar_g02_c04.avi  
  inflating: UCF-101/UCF-101/PlayingGuitar/v_PlayingGuitar_g03_c01.avi  
  inflating: UCF-101/UCF-101/PlayingGuitar/v_PlayingGuitar_g03_c02.avi  
  inflating: UCF-101/UCF-101/PlayingGuitar/v_PlayingGuitar_g03_c03.avi  
  inflating: UCF-101/UCF-101/PlayingGuitar/v_PlayingGuitar_g03_c04.avi  


In [4]:
!unzip /content/drive/MyDrive/wlasl.zip -d wlasl

Streaming output truncated to the last 5000 lines.
  inflating: wlasl/videos/43596.mp4  
  inflating: wlasl/videos/43598.mp4  
  inflating: wlasl/videos/43599.mp4  
  inflating: wlasl/videos/43671.mp4  
  inflating: wlasl/videos/43672.mp4  
  inflating: wlasl/videos/43674.mp4  
  inflating: wlasl/videos/43677.mp4  
  inflating: wlasl/videos/43679.mp4  
  inflating: wlasl/videos/43680.mp4  
  inflating: wlasl/videos/43681.mp4  
  inflating: wlasl/videos/43682.mp4  
  inflating: wlasl/videos/43684.mp4  
  inflating: wlasl/videos/43689.mp4  
  inflating: wlasl/videos/43697.mp4  
  inflating: wlasl/videos/43698.mp4  
  inflating: wlasl/videos/43700.mp4  
  inflating: wlasl/videos/43703.mp4  
  inflating: wlasl/videos/43726.mp4  
  inflating: wlasl/videos/43727.mp4  
  inflating: wlasl/videos/43729.mp4  
  inflating: wlasl/videos/43730.mp4  
  inflating: wlasl/videos/43733.mp4  
  inflating: wlasl/videos/43800.mp4  
  inflating: wlasl/videos/43801.mp4  
  inflating: wlasl/videos/43802.mp4  

In [5]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time

In [6]:
!pip install mediapipe
import mediapipe as mp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 33.0 MB 1.2 MB/s 
  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 1.12
    Uninstalling flatbuffers-1.12:
      Successfully uninstalled flatbuffers-1.12
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.9.2 requires flatbuffers<2,>=1.12, but you have flatbuffers 22.11.23 which is incompatible.


# 2. Keypoints using MP Holistic

In [7]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

In [8]:
def mediapipe_detection(image, model):
    try:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
        image.flags.writeable = False                  # Image is no longer writeable
        results = model.process(image)                 # Make prediction
        image.flags.writeable = True                   # Image is now writeable 
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    except:
      return None,None
    return image, results

In [9]:
def draw_landmarks(image, results):
    # mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACE_CONNECTIONS) # Draw face connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Draw pose connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw right hand connections

In [10]:
def draw_styled_landmarks(image, results):
    # Draw face connections
    # mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACE_CONNECTIONS, 
    #                          mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
    #                          mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
    #                          ) 
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=1, circle_radius=1), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=1, circle_radius=1)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=1, circle_radius=1), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=1, circle_radius=1)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=1, circle_radius=1), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=1, circle_radius=1)
                             ) 

# 3. Extract Keypoint Values

In [11]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    # face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, lh, rh])

# 4. Setup Folders for Collection

In [12]:
# Path for exported data, numpy arrays
ucf= 'UCF-101/UCF-101/'
wlasl='wlasl/videos/'
import os, random
from google.colab.patches import cv2_imshow
n=random.choice(os.listdir(ucf))
randvideo=random.choice(os.listdir(os.path.join(ucf,n)))
print (os.path.join(os.path.join(ucf,n),randvideo))
DATA_PATH = os.path.join('/content/') 

# Actions that we try to detect
actionsrec = np.array([wlasl,ucf])

# Thirty videos worth of data
no_sequences = 30

# Videos are going to be 30 frames in length
sequence_length = 10

# Folder start
start_folder = 30
# for action in actions: 
#     for sequence in range(no_sequences):
#         try: 
#             os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
#         except:
#             pass

UCF-101/UCF-101/HighJump/v_HighJump_g13_c05.avi


# 5. Collect Keypoint Values for Training and Testing

In [14]:
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:  
    # NEW LOOP
    # Loop through actions
    all=[]
    for action in actionsrec:
        print('the action',action)
        sequences = []
        
        if action!=wlasl:
          files = open("/content/UCF-101/TrainTestSplits-RecognitionTask/ucfTrainTestlist/trainlist01.txt")
          lists = files.readlines()
          for i in range(len(lists)):
              sequences.append(lists[i].split(" ")[0])
        else:
          sequences=os.listdir(wlasl)
        tag=action.split('/')[0]
        # Loop through sequences aka videos
        j=400
        for sequence in (sequences[0:j]):
            print(tag,os.path.join(action,sequence))
            cap = cv2.VideoCapture(os.path.join(action,sequence))
            # Loop through video length aka sequence length
            i=15
            z=0
            for frame_num in range(i):
              if cap.isOpened() or cap.read() is True:                
                ret, frame = cap.read()
                # Make detections
                image, results = mediapipe_detection(frame, holistic)
                if image is None or results is None:
                     i+=1
                     continue

                # NEW Export keypoints
                keypoints = extract_keypoints(results)
                print( os.path.exists((DATA_PATH+ 'np/'+tag+'/'+str(sequence)+str('/')+str(frame_num))))
                if not os.path.exists((DATA_PATH+ 'np/'+tag+'/'+str(sequence)+str('/'))):
                  os.makedirs(((DATA_PATH+ 'np/'+tag+'/'+str(sequence)+str('/'))))
                npy_path = ((DATA_PATH+ 'np/'+tag+'/'+str(sequence)+str('/')+str(frame_num)))
                all.append(npy_path+'.npy')  
                np.save(npy_path, keypoints)
                cv2.waitKey(100)
              else:
                i+=1

    cap.release()
    cv2.destroyAllWindows()

Streaming output truncated to the last 5000 lines.
False
False
False
False
False
False
False
False
UCF-101 UCF-101/UCF-101/ApplyEyeMakeup/v_ApplyEyeMakeup_g24_c02.avi
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
UCF-101 UCF-101/UCF-101/ApplyEyeMakeup/v_ApplyEyeMakeup_g24_c03.avi
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
UCF-101 UCF-101/UCF-101/ApplyEyeMakeup/v_ApplyEyeMakeup_g24_c04.avi
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
UCF-101 UCF-101/UCF-101/ApplyEyeMakeup/v_ApplyEyeMakeup_g24_c05.avi
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
UCF-101 UCF-101/UCF-101/ApplyEyeMakeup/v_ApplyEyeMakeup_g24_c06.avi
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
UCF-101 UCF-101/UCF-101/ApplyEyeMakeup/v_ApplyEyeMakeup_g24_c07.avi
False
False
False
False
False
False
False
F

In [43]:
cap.release()
cv2.destroyAllWindows()
[str(i).split('/')[-1] for i in (all) ]

['0.npy',
 '1.npy',
 '2.npy',
 '3.npy',
 '4.npy',
 '5.npy',
 '6.npy',
 '7.npy',
 '8.npy',
 '9.npy',
 '10.npy',
 '11.npy',
 '12.npy',
 '13.npy',
 '14.npy',
 '0.npy',
 '1.npy',
 '2.npy',
 '3.npy',
 '4.npy',
 '5.npy',
 '6.npy',
 '7.npy',
 '8.npy',
 '9.npy',
 '10.npy',
 '11.npy',
 '12.npy',
 '13.npy',
 '14.npy',
 '0.npy',
 '1.npy',
 '2.npy',
 '3.npy',
 '4.npy',
 '5.npy',
 '6.npy',
 '7.npy',
 '8.npy',
 '9.npy',
 '10.npy',
 '11.npy',
 '12.npy',
 '13.npy',
 '14.npy',
 '0.npy',
 '1.npy',
 '2.npy',
 '3.npy',
 '4.npy',
 '5.npy',
 '6.npy',
 '7.npy',
 '8.npy',
 '9.npy',
 '10.npy',
 '11.npy',
 '12.npy',
 '13.npy',
 '14.npy',
 '0.npy',
 '1.npy',
 '2.npy',
 '3.npy',
 '4.npy',
 '5.npy',
 '6.npy',
 '7.npy',
 '8.npy',
 '9.npy',
 '10.npy',
 '11.npy',
 '12.npy',
 '13.npy',
 '14.npy',
 '0.npy',
 '1.npy',
 '2.npy',
 '3.npy',
 '4.npy',
 '5.npy',
 '6.npy',
 '7.npy',
 '8.npy',
 '9.npy',
 '10.npy',
 '11.npy',
 '12.npy',
 '13.npy',
 '14.npy',
 '0.npy',
 '1.npy',
 '2.npy',
 '3.npy',
 '4.npy',
 '5.npy',
 '6.npy',


# 6. Preprocess Data and Create Labels and Features

In [44]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
ucf= 'UCF-101/UCF-101/'
wlasl='wlasl/videos/'
actions=[str(wlasl.split('/')[0]),ucf.split('/')[0]]
label_map = {label:num for num, label in enumerate(actions)}
label_map

{'wlasl': 0, 'UCF-101': 1}

In [45]:
sequences, labels = [], []
ucf= 'UCF-101/UCF-101/'
wlasl='wlasl/videos/'
# for action in actions:
i=0
for sequence in all:#np.array(os.listdir(os.path.join(DATA_PATH, action))):
        i+=1
        window = []
        print(sequence)
        # for frame_num in range(sequence_length):
        #     res = np.load(sequence)
        #     window.append(res)
        sequences.append(np.load(sequence))
        print(sequence.split('/')[3])
        if i==15:
          if sequence.split('/')[3] =='UCF-101':
            labels.append(label_map['UCF-101'])
            print(label_map['UCF-101'])
          else:
            labels.append(label_map['wlasl'])
            print(label_map['wlasl'])
          i=0
        

Streaming output truncated to the last 5000 lines.
/content/np/UCF-101/Archery/v_Archery_g17_c03.avi/11.npy
UCF-101
/content/np/UCF-101/Archery/v_Archery_g17_c03.avi/12.npy
UCF-101
/content/np/UCF-101/Archery/v_Archery_g17_c03.avi/13.npy
UCF-101
/content/np/UCF-101/Archery/v_Archery_g17_c03.avi/14.npy
UCF-101
1
/content/np/UCF-101/Archery/v_Archery_g17_c04.avi/0.npy
UCF-101
/content/np/UCF-101/Archery/v_Archery_g17_c04.avi/1.npy
UCF-101
/content/np/UCF-101/Archery/v_Archery_g17_c04.avi/2.npy
UCF-101
/content/np/UCF-101/Archery/v_Archery_g17_c04.avi/3.npy
UCF-101
/content/np/UCF-101/Archery/v_Archery_g17_c04.avi/4.npy
UCF-101
/content/np/UCF-101/Archery/v_Archery_g17_c04.avi/5.npy
UCF-101
/content/np/UCF-101/Archery/v_Archery_g17_c04.avi/6.npy
UCF-101
/content/np/UCF-101/Archery/v_Archery_g17_c04.avi/7.npy
UCF-101
/content/np/UCF-101/Archery/v_Archery_g17_c04.avi/8.npy
UCF-101
/content/np/UCF-101/Archery/v_Archery_g17_c04.avi/9.npy
UCF-101
/content/np/UCF-101/Archery/v_Archery_g17_c04.a

In [46]:
(np.array(sequences).reshape(-1,15,258)).shape
import tensorflow as tf
device = tf.test.gpu_device_name()
# device = tf.device('cuda' if tf.cuda.is_available() else 'cpu')
device

'/device:GPU:0'

In [47]:
np.array(labels).shape

(800,)

In [48]:
X = np.array((np.array(sequences).reshape(-1,15,258)))

In [49]:
X.shape

(800, 15, 258)

In [50]:
y = to_categorical(labels).astype(int)
y.shape

(800, 2)

In [51]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
# X_train = X_train.to(device)
# X_test = X_test.to(device)
# y_train = y_train.to(device)
# y_test = y_test.to(device)
# X_test.is_cuda

In [52]:
y_test.shape

(240, 2)

# 7. Build and Train LSTM Neural Network

In [53]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [54]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [55]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(15,258)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(2, activation='softmax'))

In [56]:
model#.to(device)
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [29]:
model.fit(X_train, y_train, epochs=20, callbacks=[tb_callback])

Epoch 1/20
18/18 [==============================] - 6s 73ms/step - loss: 0.4976 - categorical_accuracy: 0.6875
Epoch 2/20
18/18 [==============================] - 1s 72ms/step - loss: 0.1393 - categorical_accuracy: 0.9732
Epoch 3/20
18/18 [==============================] - 1s 70ms/step - loss: 0.0961 - categorical_accuracy: 0.9732
Epoch 4/20
18/18 [==============================] - 1s 74ms/step - loss: 0.0723 - categorical_accuracy: 0.9857
Epoch 5/20
18/18 [==============================] - 1s 75ms/step - loss: 0.0288 - categorical_accuracy: 0.9929
Epoch 6/20
18/18 [==============================] - 1s 77ms/step - loss: 0.0260 - categorical_accuracy: 0.9911
Epoch 7/20
18/18 [==============================] - 1s 76ms/step - loss: 0.7023 - categorical_accuracy: 0.8696
Epoch 8/20
18/18 [==============================] - 1s 75ms/step - loss: 0.2075 - categorical_accuracy: 0.9464
Epoch 9/20
18/18 [==============================] - 1s 76ms/step - loss: 0.0644 - categorical_accuracy: 0.9750
E

In [30]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 15, 64)            82688     
                                                                 
 lstm_1 (LSTM)               (None, 15, 128)           98816     
                                                                 
 lstm_2 (LSTM)               (None, 64)                49408     
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 2)                 66        
                                                                 
Total params: 237,218
Trainable params: 237,218
Non-trai

# 8. Make Predictions

In [31]:
res = model.predict(X_test)


8/8 [==============================] - 0s 11ms/step


In [32]:
actions[np.argmax(res[0])]

'UCF-101'

In [33]:
actions[np.argmax(y_test[0])]

'UCF-101'

In [34]:
accuracy=0
total=(X_test).shape[0]
for i in range(X_test.shape[0]):
  res = model.predict(X_test)
  if actions[np.argmax(y_test[i])]==actions[np.argmax(res[i])]:
    accuracy+=1
print(accuracy/total)


8/8 [==============================] - 0s 11ms/step
0.9958333333333333


# 9. Save Weights

In [35]:
model.save('action.h5')

In [36]:
# del model
print(X_test.shape)

(240, 15, 258)


In [37]:
model.load_weights('action.h5')

# 10. Evaluation using Confusion Matrix and Accuracy

In [38]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [39]:
yhat = model.predict(X_test)

8/8 [==============================] - 0s 9ms/step


In [40]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [41]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[121,   1],
        [  0, 118]],

       [[118,   0],
        [  1, 121]]])

In [42]:
accuracy_score(ytrue, yhat)
print ( ytrue,yhat)

[1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0] [1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0,